In [12]:
using Turing, Distributions, Random, Plots, LinearAlgebra, DelimitedFiles
using StatsFuns
using BenchmarkTools

Random.seed!(1234);
Turing.setadbackend(:forwarddiff);
Turing.setrdcache(false);

In [13]:
sunspot = readdlm("data/SN_y_tot_V2.0.csv", ';')
select_1945_2020 = filter((s) -> s[1] > 1945 && s[1] < 2021, collect(eachrow(sunspot)))
dataset = map((s) -> round(s[2]), select_1945_2020);
not_rounded = map((s) -> s[2], select_1945_2020);
N = length(dataset);


In [14]:
@model model(y) = begin
    n = length(y)
    b ~ Gamma(1000, 1/2) # Shape Scale parameterization in Distributions.jl
    z = Vector{Float64}(undef, N)
    z[1] ~ Gamma(1, 1/b) # Shape Scale parameterization in Distributions.jl
    y[1] ~ Poisson(z[1])
    for i = 2:n
        z[i] ~ Gamma(softplus(z[i-1]), 1/b)
        y[i] ~ Poisson(z[i])
    end
end

model (generic function with 2 methods)

In [15]:
# Instantiate model
sc_model = model(dataset);

In [16]:
runtime = @elapsed chain = sample(sc_model, NUTS(200, 0.65), 1000, progress=false);

┌ Info: Found initial step size
│   ϵ = 0.05
└ @ Turing.Inference /Users/mykola/.julia/packages/Turing/Suzsv/src/inference/hmc.jl:190


In [17]:
@benchmark sample(sc_model, NUTS(200, 0.65), 1000, progress=false)

┌ Info: Found initial step size
│   ϵ = 0.05
└ @ Turing.Inference /Users/mykola/.julia/packages/Turing/Suzsv/src/inference/hmc.jl:190
┌ Info: Found initial step size
│   ϵ = 0.05
└ @ Turing.Inference /Users/mykola/.julia/packages/Turing/Suzsv/src/inference/hmc.jl:190


┌ Info: Found initial step size
│   ϵ = 0.05
└ @ Turing.Inference /Users/mykola/.julia/packages/Turing/Suzsv/src/inference/hmc.jl:190
┌ Info: Found initial step size
│   ϵ = 0.05
└ @ Turing.Inference /Users/mykola/.julia/packages/Turing/Suzsv/src/inference/hmc.jl:190


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 17.748 s (8.25% GC) to evaluate,
 with a memory estimate of 25.25 GiB, over 227900207 allocations.

In [18]:
ids = findall(map(name -> occursin("z", string(name)), names(chain)));

In [19]:
mean_list = []
var_list = []
for n=2:N+1
    push!(mean_list, mean(chain[:,n,:]))
    push!(var_list, var(chain[:,n,:]))
end

In [20]:
open("estimations/nuts_sunspot_results.csv", "w") do io
    writedlm(io, [mean_list var_list])
end

In [21]:
sqrt(mean((not_rounded - mean_list).^2))

20.19020121994067